take a look at this guides:
- (Korzynski, Mazurek, Krzypkowska, & Kurasinski, 2023)

Use Template pattern for output format promtp
- (White, Fu, Hays, Sandborn, Olea, Gilbert, Elnashar, Spencer-Smith, & Schmidt, 2023)

Use PB Smells form Qamar et Al 2021 (see overleaf chapter)

Semantic Versioning: https://semverdoc.org

In [1]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate

# Initial Prompt

In [2]:
init_temp = """### Instruction ###
Act as a professional {role} working with issue trackers like Jira or Azure DevOps.
Working with these issue trackers, you must consider multiple text-based best practices to increase understandability and reduce time.
Your outputs should correspond to those a {role} would regarding the ticket.
We want to revise a ticket for compliance with these best practices and offer recommendations if needed.

### Context ###
A ticket consists of multiple fields. You are provided with the fields and a short description for each field.
Assignee: The person responsible to resolve the issue.
Comments: Community discussion on the issue, including author, timestamp, and content.
Components: Project components to which the issue belongs.
CreatedDate: The time and date the issue was created.
Creator: The person who created the issue.
Description: A detailed description of the issue.
IssueLinks: A list of links to related issues.
IssueType: The issue's purpose within the organization.
Labels: Labels to which this issue relates.
Priority: The issue importance in relation to other issues.
Project: The parent project to which the issue belongs.
Reporter: The person who found/reported the issue.
Resolution: A record of the issue's resolution, once resolved or closed.
ResolvedDate: The time and date the issue was resolved.
Status: The stage the issue is currently at in its lifecycle.
Summary: A brief one-line summary of the issue.
TimeSpent: Amount of time spent working on the issue.
VersionsAffected: The versions of the project affected by the issue.
VersionsFixed: Project versions in which the issue was (or will be) fixed.

### Best Practice ###
The best practice to check is {best_practice}"""
init_prompt = PromptTemplate(template= init_temp, input_variables= ["role", "best_practice"])

init_prompt.save("prompts/init/initPrompt_V2.1.0.json")

# Update Prompt (Best Practice)

#### Prompt

In [ ]:
update_temp ="""Best Practice: Update Missing/Outdated Fields
During the evolution of a ticket, some fields may have become outdated or missing.
You could find the needed information in the "Summary", "Description", or "Comments" fields.
Get contextual knowledge about the ticket through the "Summary", "Description", and "Comments" fields.
If someone specifically mentions or suggests a change for the ticket in the "Comments" field, update the ticket accordingly.
The change can be provided by a human or as a message from a programm.
Ignore error messages, logs and quotes within the comment.
Possible keywords for such a change are: "edit", "change", "move", "mark", "fix", "set", "update", "modify", "correct", "adjust", "resolve" and "should".
Pay particular attention to the most recent comments.

Return only mentioned or suggested changes from the "Comments" field."""

update_prompt = PromptTemplate(template= update_temp, input_variables= [])
update_prompt.save("prompts/update/updatePrompt_V1.26.0.json")

#### Output format prompt

In [ ]:
update_format_temp ="""### Instruction ###
You receive changes and recommendations made to fields of a ticket from an issue tracker.
Return the data  in the following format as JSON-format:
"<violation_predicted>": "<Set to TRUE, if a change was detected and made, otherwise FALSE>",
"fields": "<Contains the list for the receives changes, icnluding <field>, <recommendation>, and <reason>>"
"field": "<name of updated field>", 
"changed_to": "<the new input of the field after update, without any explanation>", 
"reason": "<reason for the update>"

### Context ###
Data: {revised_ticket}"""

update_format_prompt = PromptTemplate(template= update_format_temp, input_variables= ["revised_ticket"])

update_format_prompt.save("prompts/update/updateFormatPrompt_V1.12.0.json")

# Summary Length Prompt (Best Practice)


examples:
- https://bugzilla.mozilla.org/page.cgi?id=bug-writing.html
- https://bugs.eclipse.org/bugs/page.cgi?id=bug-writing.html

#### Zero-Shot Prompt

In [5]:
sumLen_temp = """'Summary Length':
Your Task is to check if the ticket summary has exactly between {min} and {max} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.

The ticket is in JSON format.
Ticket: {ticket}"""

sumLen_prompt = PromptTemplate(template= sumLen_temp, input_variables= ["min", "max", "ticket"])

sumLen_prompt.save("prompts/summary/summaryLengthPrompt_V3.2.0.json")

#### Zero-Shot CoT Prompt

In [2]:
sumLen_0ShotCoT_temp = """Best Practice: Summary Length
Your Task is to check if the ticket summary has exactly between {min} and {max} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.

The ticket is in JSON format.
Ticket: {ticket}
Answer: Let's think step by step."""

sumLen_0ShotCoT_prompt = PromptTemplate(template= sumLen_0ShotCoT_temp, input_variables= ["min", "max", "ticket"])

sumLen_0ShotCoT_prompt.save("prompts/summary/summaryLengthPrompt_0ShotCoT_V2.1.0.json")

#### Few-Shot Prompt

In [3]:
examples = [
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Sourcetree crashes',
'Description':'Source tree crashes everytime i open , I have uninstalled and installed many times',
'VersionsAffected':'3.4.4',
'IssueType':'Bug',
'Project':'Sourcetree for Windows',
'Components':'Git'}}""",
        "output": """The current summary is "Sourcetree crashes", which consists of 18 characters. This is below the recommended range of 39 to 70 characters for a summary.

Recommended Summary: "Sourcetree crashes upon opening repeatedly"

This revised summary is 42 characters long, fitting within the recommended range and succinctly conveying the essence of the ticket."""
    },
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Opening and closing Sourcetree causes Pageant to crash\/close',
'Description':'I have installed putty and pageant outside of Sourcetree, and I have Pageant startup during OS boot so my key-file is always ready.\r\n\r\nDidn't quite notice it at first, so not sure which version it started at, but Sourcetree now causes Pageant to crash\/close when it opens or closes, causing e.g. git commands and such from command line to suddenly start to failing for no apparent reason.\r\n\r\nWhenever I open or close Sourcetree now, I have to reopen Pageant and reload my key, including entering a long passkey. Super annoying. Sourcetree shouldn't close or cause any other applications to crash when it opens or when it closes.\r\n\r\nIt happens on all 3 computers of mine. All running latest Windows, latest Pageant, and latest Sourcetree.',
'VersionsAffected':'3.4.4',
'IssueType':'Bug',
'Project':'Sourcetree for Windows',
'Components':'General'}}""",
        "output": """The current summary is "Opening and closing Sourcetree causes Pageant to crash\/close" which consists of 61 characters. This is within the best practice range of 39 to 70 characters.

No summary has to be recommended. 

The original summary is compliant with the best practice for summary length and does not require any changes."""
    },
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Cannot change title of News tool',
'Description':'After initially setting up the News tool during worksite setup, I tried to go to the option page (NEWS-2) and change the title of the tool. After I made the change, the title of the tool did not update, even after I logged out and logged back in. However, when I opened up options, the new title that I had entered was still specified.\n\nI tried to change the URL of the feed as well as the title at the same time, and even though the new feed was displayed properly, the title of the tool remained what I had initially set it to during worksite setup.',
'VersionsAffected':'2.0',
'IssueType':'Bug',
'Project':'Sakai',
'Components':'News (RSS)'}}""",
        "output": """The current summary is "Cannot change title of News tool", which consists of 32 characters. This is below the recommended range of 39 to 70 characters for a summary.

Recommended Summary: "Title of News tool remains unchanged after several update attempts"

This revised summary is 66 characters long, fitting within the recommended range and succinctly conveying the essence of the ticket."""
    },
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Game Crashes When Player Opens Inventory While Respawning with 'Immediate Respawn' Enabled',
'Description':'If the player opens their inventory quickly after respawning in a world with 'Immediate Respawn' enabled, the world will crash. This happened accidentally once in one of my worlds. I tried it a few more times after reloading, and it seemed to happen reliably as long as I opened my inventory at the right time.',
'VersionsAffected':'PS4 1.89',
'IssueType':'Bug',
'Project':'Minecraft Console Edition',
'Components':null}}""",
        "output": """The current summary is "Game Crashes When Player Opens Inventory While Respawning with 'Immediate Respawn' Enabled", which consists of 90 characters. This exceeds the recommended range of 39 to 70 characters for a summary.

Recommended Summary: "Crash on inventory access with active 'Immediate Respawn' on respawn"

This revised summary is 68 characters long, fitting within the recommended range and succinctly conveying the essence of the ticket."""
    },
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Epic link gets removed when moving an epic from a non next gen project to a next project',
'Description':'h3. Summary \r\n\r\n When an epic from a non next-Gen project is moved to a next-Gen project, the association is removed. All issues associated with the epics will  no longer be linked to the epic \r\n\r\n h3. Steps to reproduce \r\n\r\n # Have 2 projects (Next-gen and non next-gen) \r\n # Create an Epic from a non Next-gen project \r\n # Add a child issue to the epic \r\n # Move the epic to the next-gen project \r\n\r\n h3. Expected Result \r\n Epic link remains. The issues are still linked with the epic in the next-gen project even after the moved. \r\n\r\n h3. Actual Result \r\n Epic link is lost',
'VersionsAffected':null,
'IssueType':'Bug',
'Project':'Jira Software Cloud',
'Components':'next-gen'}}""",
        "output": """The current summary is "Epic link gets removed when moving an epic from a non next gen project to a next project", which consists of 88 characters. This exceeds the recommended range of 39 to 70 characters for a summary.

Recommended Summary: "Epic link loss on moving an epic to a 'next-gen' project"

This revised summary is 56 characters long, fitting within the recommended range and succinctly conveying the essence of the ticket."""
    }
]

In [4]:
sumLen_FewShot_temp = PromptTemplate(
    input_variables=["minChar", "maxChar", "input", "output"], 
    template="""'Summary Length':
Your Task is to check if the ticket summary has exactly between {minChar} and {maxChar} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.

The ticket is in JSON format.
Ticket: {input}
Answer: {output}"""
)

In [5]:
sumLen_FewShot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=sumLen_FewShot_temp,
    suffix= """'Summary Length':
Your Task is to check if the ticket summary has exactly between {min} and {max} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.

The ticket is in JSON format.
Ticket: {ticket}
Answer: """,
    input_variables=["min", "max", "ticket"],
)

sumLen_FewShot_prompt.save("prompts/summary/summaryLengthPrompt_FewShot_V2.3.0.json")

#### Few-Shot CoT Prompt

In [5]:
examples = [
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Sourcetree crashes',
'Description':'Source tree crashes everytime i open , I have uninstalled and installed many times',
'VersionsAffected':'3.4.4',
'IssueType':'Bug',
'Project':'Sourcetree for Windows',
'Components':'Git'}}""",
        "output": """The current summary is "Sourcetree crashes". Splitting the summary into it's individual characters, we get the following list: ['S', 'o', 'u', 'r', 'c', 'e', 't', 'r', 'e', 'e', ' ', 'c', 'r', 'a', 's', 'h', 'e', 's'].
Counting each character including all letters, spaces, and punctuation marks, we get a total of 18 characters. This is below the recommended range of 39 to 70 characters for a summary, so a new summary has to be recommended.

Recommended Summary: "Sourcetree crashes upon opening repeatedly"

This revised summary is 42 characters long, fitting within the recommended range and succinctly conveying the essence of the ticket."""
    },
    {
        "minChar": 39,
        "maxChar": 70,
        "input": """{{'Summary':'Opening and closing Sourcetree causes Pageant to crash\/close',
'Description':'I have installed putty and pageant outside of Sourcetree, and I have Pageant startup during OS boot so my key-file is always ready. Didn't quite notice it at first, so not sure which version it started at, but Sourcetree now causes Pageant to crash\/close when it opens or closes, causing e.g. git commands and such from command line to suddenly start to failing for no apparent reason. Whenever I open or close Sourcetree now, I have to reopen Pageant and reload my key, including entering a long passkey. Super annoying. Sourcetree shouldn't close or cause any other applications to crash when it opens or when it closes. It happens on all 3 computers of mine. All running latest Windows, latest Pageant, and latest Sourcetree.',
'VersionsAffected':'3.4.4',
'IssueType':'Bug',
'Project':'Sourcetree for Windows',
'Components':'General'}}""",
        "output": """The current summary is "Opening and closing Sourcetree causes Pageant to crash\/close". Splitting the summary into it's individual characters, we get the following list: ['O', 'p', 'e', 'n', 'i', 'n', 'g', ' ', 'a', 'n', 'd', ' ', 'c', 'l', 'o', 's', 'i', 'n', 'g', ' ', 'S', 'o', 'u', 'r', 'c', 'e', 't', 'r', 'e', 'e', ' ', 'c', 'a', 'u', 's', 'e', 's', ' ', 'P', 'a', 'g', 'e', 'a', 'n', 't', ' ', 't', 'o', ' ', 'c', 'r', 'a', 's', 'h', '\', '/', 'c', 'l', 'o', 's', 'e'].
Counting each character including all letters, spaces, and punctuation marks, we get a total of 61 characters. This is within the best practice range of 39 to 70 characters.    
        
No summary has to be recommended. 

The original summary is compliant with the best practice for summary length and does not require any changes."""
     },
#     {
#         "minChar": 39,
#         "maxChar": 70,
#         "input": """{{'Summary':'Game Crashes When Player Opens Inventory While Respawning with 'Immediate Respawn' Enabled',
# 'Description':'If the player opens their inventory quickly after respawning in a world with 'Immediate Respawn' enabled, the world will crash. This happened accidentally once in one of my worlds. I tried it a few more times after reloading, and it seemed to happen reliably as long as I opened my inventory at the right time.',
# 'VersionsAffected':'PS4 1.89',
# 'IssueType':'Bug',
# 'Project':'Minecraft Console Edition',
# 'Components':null}}""",
#         "output": """The current summary is "Game Crashes When Player Opens Inventory While Respawning with 'Immediate Respawn' Enabled". Splitting the summary into it's individual characters, we get the following list: ['G', 'a', 'm', 'e', ' ', 'C', 'r', 'a', 's', 'h', 'e', 's', ' ', 'W', 'h', 'e', 'n', ' ', 'P', 'l', 'a', 'y', 'e', 'r', ' ', 'O', 'p', 'e', 'n', 's', ' ', 'I', 'n', 'v', 'e', 'n', 't', 'o', 'r', 'y', ' ', 'W', 'h', 'i', 'l', 'e', ' ', 'R', 'e', 's', 'p', 'a', 'w', 'n', 'i', 'n', 'g', ' ', 'w', 'i', 't', 'h', ' ', '\'', 'I', 'm', 'm', 'e', 'd', 'i', 'a', 't', 'e', ' ', 'R', 'e', 's', 'p', 'a', 'w', 'n', '\'', ' ', 'E', 'n', 'a', 'b', 'l', 'e', 'd'].
# Counting each character including all letters, spaces, and punctuation marks, we get a total of 90 characters. This exceeds the recommended range of 39 to 70 characters for a summary, so a new summary has to be recommended.

# Recommended Summary: "Crash on inventory access with active 'Immediate Respawn' on respawn"

# This revised summary is 68 characters long, fitting within the recommended range and succinctly conveying the essence of the ticket."""
#     }
]

In [6]:
sumLen_CoT_temp = PromptTemplate(
    input_variables=["minChar", "maxChar", "input", "output"], 
    template="""'Summary Length':
Your Task is to check if the ticket summary has exactly between {minChar} and {maxChar} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.

The ticket is in JSON format.
Ticket: {input}
Answer: Let's think step by step. {output}"""
)

In [7]:
sumLen_CoT_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=sumLen_CoT_temp,
    suffix= """'Summary Length':
Your Task is to check if the ticket summary has exactly between {min} and {max} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.

The ticket is in JSON format.
Ticket: {ticket}
Answer: Let's think step by step.""",
    input_variables=["min", "max", "ticket"],
)

sumLen_CoT_prompt.save("prompts/summary/summaryLengthPrompt_CoT_V1.2.0.json")

#### Output Format Prompt

In [6]:
sumLen_format_temp = """### Instruction ###
You receive a revised ticket from an issue tracker in JSON format.
Return the given Data in the following format in JSON:
"<violation_predicted>": "<Set to TRUE, if a change was detected and made, otherwise FALSE>"
"<summary_old>": "<The current and unchanged content of the summary field>"
"<summary_new>": "<The recommended and revised content of the summary field>"

If the summary was not changed, return the original and unchanged summary in the "<summary_new>" field.
Don't return any source code or code snippets.

### Context ###
Data: {revised_ticket}"""

sumLen_format_prompt = PromptTemplate(template= sumLen_format_temp, input_variables= ["revised_ticket"])

sumLen_format_prompt.save("prompts/summary/summaryLengthFormatPrompt_V1.5.0.json")

#### Consensus Prompt

In [7]:
sumLen_consensus_temp = """### Instruction ###
Act as a professional {role} working with issue trackers like Jira or Azure DevOps.
Working with these issue trackers, you must consider multiple text-based best practices to increase understandability and reduce time.
Your outputs should correspond to those a {role} would regarding the ticket.

Previously, several prompt algorithms were tested for the best practice 'Summary Length'. This best practice states:
Your Task is to check if the ticket summary has exactly between {min} and {max} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary that number of characters is exactly in this range based on the information of the description field for the ticket.

### Context ###
Data: {ticket}

Given the following answers, which one is the best, or can you synthesize a better one?
Return the prompt type which provides the best answer with a short one sentence explanation.
{answers}"""

sumLen_consensus_prompt = PromptTemplate(template= sumLen_consensus_temp, input_variables= ["ticket", "role", "min", "max", "answers"])

sumLen_consensus_prompt.save("prompts/summary/summaryLengthConsensusPrompt_V1.0.0.json")

In [4]:
sumLen_consensus_form_temp = """### Instruction ###
You receive the prompt type, the answer, the new summary and an explanation, regarding the questions of which of a set of summarys is the best.
Return the given Data in the following format in JSON:
"<prompt_type>": "<The prompt type, or NEW if not provided>"
"<summary>": "<The new and recommended summary>"
"<explanation>": "<The short explanation>"

### Context ###
Data: {answer}"""

sumLen_consensus_form_prompt = PromptTemplate(template= sumLen_consensus_form_temp, input_variables= ["answer"])

sumLen_consensus_form_prompt.save("prompts/summary/summaryLengthConsensusFormatPrompt_V1.2.0.json")

# Arbitrary Structure Prompt (Best Practice)

#### Prompt

"""Best Practice: Description Completeness
The received ticket is from type "{issue_type}".
Your Task is to check if the "Description" field of the ticket is complete.
We define a description as complete if it contains the following pieces:
{content}
Role: abstract behavior of actors in the system context; describes who uses the system.
Task: specific things that must be done to achieve goals; solution or function of the problem.
Capability: the ability of actors to achieve goals based on certain conditions and events.
Goal: a condition or a circumstance desired by stakeholders or actors; describes the problem domain or the impact of solving the problem.

The general format of a {issue_type} can be expressed as follows: 
{strucutre}
As a <Role>, I want <Task>, So that <Goal>.

If the description is complete, return the original description.
Otherwise, recommend a new description for the ticket.
"""

In [ ]:
descComp_temp = """Best Practice: Arbitrary Structure
The received ticket is from the superior type "{issue_type}".
Your Task is to check if the "Description" field of the ticket contains all components of a given structure.
The components of the desired structure are described as follows:
{structure_desc}

The desired strucutre of a {issue_type} is as follows: 
{strucutre}

If the description is conatins all components, return the original description.
If the description contains nothing, return "No description provided.".
Otherwise, recommend a new description, suitable to the desired structure for the ticket.
"""

descComp_prompt = PromptTemplate(template= descComp_temp, input_variables= ["issue_type", "structure_desc", "structure"])

descComp_prompt.save("prompts/arbitraryStructure/arbitraryStructurePrompt_V1.0.0.json")

#### Output format prompt

In [ ]:
descComp_format_temp = """### Instruction ###
You receive a revised ticket from an issue tracker in JSON format.
Return the given Data in the following format in JSON:
"<violation_predicted>": "<TRUE, if a violation was detectd, otherwise FALSE>"
"<description_old>": "<Old version of the content of the description field>"
"<description_new>": "<New version of the content of the description field>"

If the description was not changed, return the original description in the "<description_new>" field.
Don't return any source code or code snippets.

### Context ###
Data: {revised_ticket}"""

descComp_format_prompt = PromptTemplate(template= descComp_format_temp, input_variables= ["revised_ticket"])

descComp_format_prompt.save("prompts/arbitraryStructure/arbitraryStructureFormatPrompt_V1.0.0.json")

# Toxic Speech Prompt (Best Practice)

#### Prompt

we find that including the term “ethnic slur” in our prompt greatly improves the performance because this term specifies the victims to be a group. Using
“offensive or toxic content” as prompt doesn’t indi- cate the target of the victim, and thus leads to poor performance. 

LLMs cannot understand “hatespeech” is targeting a specific group, and thus also leads to poor performance.

it seems that explic- itly saying “intend to offend" is useful in surfacing out the LLMs’ underlying self-diagnosing abilities

(Wang & Chang 2022)

In [ ]:
toxicSpeech_temp = """Best Practice: Toxic Speech Detection
Your task is to check the comments section for toxic speech.
If you detect toxic speech, return the comment, the author, the datetime, and give a recommendation how to updated the comment."""

toxicSpeech_prompt = PromptTemplate(template= toxicSpeech_temp, input_variables= [])

toxicSpeech_prompt.save("prompts/toxicSpeech/toxicSpeechPrompt_V1.5.0.json")

#### Output format prompt

In [ ]:
toxicSpeech_format_temp = """### Instruction ###
You receive a list of comments from a revised ticket from an issue tracker.
Return the given Data for each entry in the following format in JSON:
"comment": "<content of the original comment>",
"author": "<name of the original comment>",
"datetime": "<datetime of the original comment>",
"recommendation": "<only the revised string of the new comment>"
"reason": "<reason for the recommendation>"
If the list is empty return "no toxic speech detected".

### Context ###
Data: {revised_ticket} """

toxicSpeech_format_prompt = PromptTemplate(template= toxicSpeech_format_temp, input_variables= ["revised_ticket"])

toxicSpeech_format_prompt.save("prompts/toxicSpeech/toxicSpeechFormatPrompt_V1.2.0.json")

# Internationalization Prompt (Best Practice)

#### Prompt

In [ ]:
# Sprache variable machen
internationalization_temp = """Best Practice: Internationalization
Bug reports not written in English are often closed immediately or ignored by developers.
Your task is to check if the "Summary", "Description", or "Comments" fields of the ticket are written in English.
If some fields of the ticket are not written in English, provide a translation.
Return only the revised ticket and nothing else."""

internationalization_prompt = PromptTemplate(template= internationalization_temp, input_variables= [])

internationalization_prompt.save("prompts/internationalization/internationalizationPrompt_V1.5.0.json")

#### output format prompt

In [ ]:
internationalization_format_temp = """### Instruction ###
You receive a revised ticket from an issue tracker in JSON format.
Return the given Data in the following format in JSON:
"<field>": "<content of the field>"

### Context ###
Data: {revised_ticket}"""

internationalization_format_prompt = PromptTemplate(template= internationalization_format_temp, input_variables= ["revised_ticket"])

internationalization_format_prompt.save("prompts/internationalization/internationalizationFormatPrompt_V1.2.0.json")

# Bug Report Structure Prompt

maybe more of this structure:
- https://bugzilla.mozilla.org/page.cgi?id=bug-writing.html
- https://bugs.eclipse.org/bugs/page.cgi?id=bug-writing.html
- https://help.minecraft.net/hc/en-us/articles/4408887473421-Mojang-Bug-Tracker-Guidelines-and-FAQ#h_01FF6B0GPYQJR44QAMJN7YEXXR


#### 0-Shot Prompt

In [2]:
bugReportStructure_temp = """'Bug Report Structure':
The description field is important for the bug report, as it describes the issue in detail.

### Description Components ###
The description of a bug report should contain the following important components:
Overview: A more detailed description of the context when the error occurred.
Steps to reproduce: A list of steps to reproduce the error.
Actual result: The result that is currently produced.
Expected result: The result that is expected.

Additionally, the description could contain the following components:
Stack Traces: A list of relevant stack traces for the bug report.
Build date and hardware: The build date and hardware, as well as when and where the bug occurred.
Additional information: Additional information relevant to the bug report (screenshots, videos, or other files).

### Task ###
We want to determine the degree of completeness of the original/old description.
The important components of a description are an overview/summary, steps to reproduce, the actual result and the expected result.
The degree is between 1 and 3:
1: The original/old description contains all important components or has only been slightly modified.
2: The original/old description contains 2 or 3 components but is not complete.
3: The original/old description is only a text without a clear structure and contains at most  1 important component.

If the description field is missing some important parts, we want to rearrange the description field accordingly.
Return the degree of completeness, the revised description, and a short two-sentence explanation for the changes made/not made.

The bug report is in JSON format.
Bug report: {bug_report}"""

bugReportStructure_prompt = PromptTemplate(template=bugReportStructure_temp, input_variables=["bug_report"])

bugReportStructure_prompt.save("prompts/bugReportStructure/bugReportStructurePrompt_V2.1.0.json")

#### Zero-Shot CoT Prompt

In [2]:
bugReportStructure_0ShotCoT_temp = """'Bug Report Structure':
The description field is important for the bug report, as it describes the issue in detail.

### Description Components ###
The description of a bug report should contain the following important components:
Overview: A more detailed description of the context when the error occurred.
Steps to reproduce: A list of steps to reproduce the error.
Actual result: The result that is currently produced.
Expected result: The result that is expected.

Additionally, the description could contain the following components:
Stack Traces: A list of relevant stack traces for the bug report.
Build date and hardware: The build date and hardware, as well as when and where the bug occurred.
Additional information: Additional information relevant to the bug report (screenshots, videos, or other files).

### Task ###
We want to determine the degree of completeness of the original/old description.
The important components of a description are an overview/summary, steps to reproduce, the actual result and the expected result.
The degree is between 1 and 3:
1: The original/old description contains all important components or has only been slightly modified.
2: The original/old description contains 2 or 3 components but is not complete.
3: The original/old description is only a text without a clear structure and contains at most 1 important component.

If the description field is missing some important parts, we want to rearrange the description field accordingly.
Return the degree of completeness, the revised description, and a short two-sentence explanation for the changes made/not made.

The bug report is in JSON format.
Bug report: {bug_report}
Answer: Let's think step by step."""

bugReportStructure_0ShotCoT_prompt = PromptTemplate(template= bugReportStructure_0ShotCoT_temp, input_variables= ["bug_report"])

bugReportStructure_0ShotCoT_prompt.save("prompts/bugReportStructure/bugReportStructurePrompt_0ShotCoT_V1.0.0.json")

#### Few-Shot Prompt

In [2]:
examples = [
    {
        "input":"""{{'Summary':'Approvals history items with deleted status' are breaking the Bento view',
'Description':'Approvals history items with deleted status' are breaking the Bento view\r\nh3. Steps to Reproduce\r\n# Create an approval status on your workflow.\r\n# Transition the ticket and approve it.\r\n# Now, delete the status created on the first step.\r\n\r\nh3. Expected Results\r\nYou should be able to access the ticket in the new issue view.\r\n\r\nh3. Actual Results\r\n\r\nThe ticket does not load as the ticket history points to the deleted status and an exception is thrown in the logs.\r\n\r\nh3. Workaround\r\nAccess the ticket via the new issue view or clone the ticket.'}}""",
        "output": """Degree of Completeness: 1

Revised Description:
```
Overview
Approvals history items with deleted status' are breaking the Bento view.

Steps to reproduce
1. Create an approval status on your workflow.
2. Transition the ticket and approve it.
3. Now, delete the status created on the first step.

Actual result
The ticket does not load as the ticket history points to the deleted status and an exception is thrown in the logs.

Expected result
You should be able to access the ticket in the new issue view.

Additional information
a) Workaround: Access the ticket via the new issue view or clone the ticket.
```

Explanation: The description starts with an overview, even if it does not have a corresponding heading. The degree of completeness is therefore 1 as the original description contains all the important parts and has only been slightly restructured."""
    },
    {
        "input": """{{'Summary':'Text formatting does not work in the Visual mode of the Comment field in a Transition screen',
'Description':'h3. Problem definition\r\nText formatting like bold, italic, underline and color do not work for the Comment added in the Visual mode of a transition screen.\r\n\r\nh3. Steps to reproduce the issue\r\n# Configure a screen for any transition for a workflow associated to a Service Desk project.\r\n# Ensure that the *Rich text editor* is enabled.\r\n# Transition a Service Desk issue and on the pop-up screen, use the *Visual mode*.\r\n# Type something and make it bold\/italic\/underlined\/colored.\r\n\r\nh3. Expected result\r\nThe comment will be shown with the text formatting applied.\r\n\r\nh3. Actual result\r\nThe comment is highlighted without any text formatting applied.\r\n\r\nh3. Notes\r\n# This does not happen on issue view, only transition screen.\r\n# This does not happen to non-Service Desk projects.'}}""",
        "output": """Degree of Completeness: 1

Revised Description:
```
Overview
Text formatting like bold, italic, underline and color do not work for the Comment added in the Visual mode of a transition screen.

Steps to reproduce
1. Configure a screen for any transition for a workflow associated to a Service Desk project.
2. Ensure that the *Rich text editor* is enabled.
3. Transition a Service Desk issue and on the pop-up screen, use the *Visual mode*.
4. Type something and make it bold\/italic\/underlined\/colored.

Actual result
The comment is highlighted without any text formatting applied.

Expected result
The comment will be shown with the text formatting applied.

Additional information
a) This does not happen on issue view, only transition screen.
b) This does not happen to non-Service Desk projects.
```

Explanation: The original description contains all the important components, including the overview, steps to reproduce, the actual result, and the expected result. The degree of completeness is therefore 1 as the original description contains all important parts including the topics for separation and has only been slightly restructured."""
    },
    {
        "input":"""{{'Summary':'Cmake warning spam in build of maint-4952-v-t-u (coroutine)',
'Description':'h1. Steps to Reproduce\nBuilding opensource maint-4952-v-t-u \n\nh1. Actual Behavior\nBuilt HTTPcoroutine on windows and it generated a lot of cmake warnings mostly boost in cef.\n\nh1. Expected Behavior\nNot have the warnings \n\nh1. Other information\nThis warning is for project developers.  Use -Wno-dev to suppress it.\r\nI could add -Wno-dev. But I think these warnings should be corrected.'}}""",
        "output": """Degree of Completeness: 2

Revised Description:
```
Overview
During the build of opensource maint-4952-v-t-u, a lot of CMake warnings are generated. 

Steps to reproduce
1. Building opensource maint-4952-v-t-u

Actual result
Built HTTPcoroutine on windows and it generated a lot of cmake warnings mostly boost in cef.

Expected result
Not have the warnings

Additional information
a) This warning is for project developers. 
b) Use -Wno-dev to suppress it. But I think these warnings should be corrected.
```

Explanation: The original description starts directly with the steps to reproduce, followed by the actual result and the expected result, but there are no indications for the overview. The degree of completeness is therefore 2 as the missing overview component has to be generated."""
    },
    {
        "input":"""{{'Summary':'its still demo',
'Description':'i just buy paceofcarde for 100 zl becose i want to buy minecraft but when i go to the webside and i dowlond the game it show that is demo i tried to dowlond one more time i chec i this the same username and password but its stil nothing pleas help'}}""",
        "output": """Degree of Completeness: 3
        
Revised Description:
```
Overview
I want to buy minecraft, but after the download it shows that it is a demo version. I already checked the username and password and repeated the download but it still is the demo version.

Steps to reproduce
1. Go to the minecraft website
2. Download the game

Actual result
The demo version of the game is downloaded

Expected result
The full version of the game should be downloaded
```

Explanation: The original description is plain text with an explanation of what the person did, but no structure was given. The degree of completeness is therefore 3 because the original description was a simple text without any headings or structure."""
    },
    {
        "input":"""{{'Summary':'Exceptions raised during AM launch don't trigger exit code AM failures',
'Description':'It looks like exceptions raised in AM launch failure, while being caught, don't have their exit code picked up. Any launch-time failure should result in an exit code being returned, which then tells the YARN to restart it.'}}""",
        "output": """Degree of Completeness: 3
        
Revised Description:
```
Overview
The exit code of exceptions raised in AM launch failure is not picked up. Even if the exception is caught correctly.

Steps to reproduce
1. Launch AM
2. Observe the exceptions raised during the launch

Actual result
Exceptions raised in AM launch failure, don't have their exit code picked up, while being caught.

Expected result
Any launch-time failure should result in an exit code being returned, which then tells the YARN to restart it.
```

Explanation: Even if the original description assumes that it contains the expected behavior and an overview, there is no clear structure through topics. The degree of completeness is therefore 3 as the original description was a plain text without any headings or structure."""
    }
]

In [3]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"], 
    template="""Bug Report: {input}

Answer: {output}"""
)

In [4]:
bugReportStructure_examples_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix= """Bug report: {bug_report}

Answer: """,
    input_variables=["bug_report"]
)

bugReportStructure_examples_prompt.save("prompts/bugReportStructure/bugReportStructurePrompt_FewShot_Examples_V1.2.0.json")

In [5]:
bugReportStructure_task_temp = """'Bug Report Structure':
The description field is important for the bug report, as it describes the issue in detail.

### Description Components ###
The description of a bug report should contain the following important components:
Overview: A description of the context/problem, a summary, te requirement, or a problem statement. This is usual the beginning of the description. Keywords for overview heading are: overview, summary, description, problem;
Steps to reproduce: A list of steps to reproduce, or the scenario to produce the error. Keywords for steps to reproduce heading are: steps, reproduce, scenario, how to;
Actual result: The result that is currently produced, the actual behaviour, or what actual happens. Keywords for actual result heading are: actual, result, behaviour, happens;
Expected result: The result that was expected, the expected behaviour, or what should happen. Keywords for expected result heading are: expected, result, behaviour, should.

Additionally, the description could contain the following components:
Stack Traces: A list of relevant stack traces for the bug report.
Build date and hardware: The build date and hardware, as well as when and where the bug occurred.
Additional information: Additional information relevant to the bug report, including screenshots, videos, or other files.

### Task ###
We want to determine the degree of completeness of the original/old description based on the provided context of the description field.
The important components of a description are an overview, steps to reproduce, the actual result and the expected result.
We define a clear structure when we have appropriate headings for each important component in the description, except for the overview component.
Even if the overview component does not have a heading, but can be assumed as overview it is considered to be present and complete.
The degree is between 1 and 3:
1: The original/old description contains all important components or has only been rearranged.
2: The original/old description contains a minimum of 2 components but is not complete.
3: The original/old description is a plain text without a clear structure.

If the description field is missing some important parts, we want to rearrange the description field accordingly.
Return the degree of completeness, the revised description, and a short two-sentence explanation for the changes made.

The bug report is in JSON format.
{examples}"""

bugReportStructure_task_prompt = PromptTemplate(template=bugReportStructure_task_temp, input_variables=["examples"])

bugReportStructure_task_prompt.save("prompts/bugReportStructure/bugReportStructurePrompt_FewShot_V2.11.0.json")

#### Few-Shot CoT

In [2]:
examples = [
    {
        "input":"""{{'Summary':'Cmake warning spam in build of maint-4952-v-t-u (coroutine)',
'Description':'Steps to Reproduce\nBuilding opensource maint-4952-v-t-u \n\nActual Behavior\nBuilt HTTPcoroutine on windows and it generated a lot of cmake warnings mostly boost in cef.\n\nExpected Behavior\nNot have the warnings \n\nOther information\nThis warning is for project developers.  Use -Wno-dev to suppress it.\r\nI could add -Wno-dev. But I think these warnings should be corrected.'}}""",
        "output": """We want to check if all important components are present in the description field of the ticket.
First we want to check if an overview is present. There is no keyword or heading that indicates an overview. The description starts directly with the steps to reproduce. The overview component is missing.
Secondly, we want to check if the steps to reproduce are present. The steps to reproduce are clearly defined under the heading "Steps to Reproduce". Corresponding steps are listed. The steps to reproduce component is present.
Thirdly, we want to check if the actual result is present. The actual result is clearly defined under the heading "Actual Behavior". The actual result component is present.
Lastly, we want to check if the expected result is present. The expected result is clearly defined under the heading "Expected Behavior". The expected result component is present.
Additionally, we have the component "Additional information" highlighted by the header "Other information". 

Three important components are available, but there is no overview, so the Degree of Completeness is: 2 

Revised Description:
```
Overview
During the build of opensource maint-4952-v-t-u, a lot of CMake warnings are generated. 

Steps to reproduce
1. Building opensource maint-4952-v-t-u

Actual result
Built HTTPcoroutine on windows and it generated a lot of cmake warnings mostly boost in cef.

Expected result
Not have the warnings

Additional information
a) This warning is for project developers. 
b) Use -Wno-dev to suppress it. But I think these warnings should be corrected.
```

Explanation: The original description starts directly with the steps to reproduce, followed by the actual result and the expected result, but there are no indications for the overview. The degree of completeness is therefore 2 as the missing overview component has to be generated."""
    },
    {
        "input":"""{{'Summary':'Approvals history items with deleted status' are breaking the Bento view',
'Description':'Approvals history items with deleted status' are breaking the Bento view\r\nh3. Steps to Reproduce\r\n# Create an approval status on your workflow.\r\n# Transition the ticket and approve it.\r\n# Now, delete the status created on the first step.\r\n\r\nh3. Expected Results\r\nYou should be able to access the ticket in the new issue view.\r\n\r\nh3. Actual Results\r\n\r\nThe ticket does not load as the ticket history points to the deleted status and an exception is thrown in the logs.\r\n\r\nh3. Workaround\r\nAccess the ticket via the new issue view or clone the ticket.'}}""",
        "output": """We want to check if all important components are present in the description field of the ticket.
First, we want to check if an overview is present. No keyword or heading indicates an overview, but the first sentence of the description can be interpreted as an overview by the ticket context. The overview component is present.
Secondly, we want to check if the steps to reproduce are present. The steps to reproduce are clearly defined under the heading "Steps to Reproduce," which is also recognizable by its h1.-heading. Corresponding steps are listed. The steps to reproduce component is present.
Thirdly, we want to check if the actual result is present. The actual result is clearly defined under the heading "Actual Results" which is also recognizable by its h1.-heading. The actual result component is present.
Lastly, we want to check if the expected result is present. The expected result is clearly defined under the heading "Expected Results", which is also recognizable by its h1.-heading. The expected result component is present.
        
All important components are available so the Degree of Completeness is: 1

Revised Description:
```
Overview
Approvals history items with deleted status' are breaking the Bento view.

Steps to reproduce
1. Create an approval status on your workflow.
2. Transition the ticket and approve it.
3. Now, delete the status created on the first step.

Actual result
The ticket does not load as the ticket history points to the deleted status and an exception is thrown in the logs.

Expected result
You should be able to access the ticket in the new issue view.

Additional information
a) Workaround: Access the ticket via the new issue view or clone the ticket.
```

Explanation: The descriptions contains an overview, steps to reproduce, an actual result and an expected result so all important components are included. The degree of completeness is therefore 1 as the original description contains all the important parts and has only been slightly restructured."""
    },
    {
        "input":"""{{'Summary':'Exceptions raised during AM launch don't trigger exit code AM failures',
'Description':'It looks like exceptions raised in AM launch failure, while being caught, don't have their exit code picked up. Any launch-time failure should result in an exit code being returned, which then tells the YARN to restart it.'}}""",
        "output": """We want to check if all important components are present in the description field of the ticket.
First, we want to check if an overview is available. There is no keyword or heading that indicates an overview. The whole description is plain text describing a problem, which could be interpreted as the overview component. The overview component is present.
Secondly, we want to check if the steps to reproduce are present. There is no keyword or heading that indicates the steps to reproduce. The steps to reproduce component is missing.
Thirdly, we want to check if the actual result is present. No keyword or heading indicates the actual result, even if a potential result can be inferred from the overview text. The actual result component is missing.
Lastly, we want to check if the expected result is present. No keyword or heading indicates the expected result, even if a potential expected result can be inferred from the overview text. The expected result component is missing.

Besides a plain text as the overview no structure was provided and important components are missing so the Degree of Completeness is: 3
        
Revised Description:
```
Overview
The exit code of exceptions raised in AM launch failure is not picked up. Even if the exception is caught correctly.

Steps to reproduce
1. Launch AM
2. Observe the exceptions raised during the launch

Actual result
Exceptions raised in AM launch failure, don't have their exit code picked up, while being caught.

Expected result
Any launch-time failure should result in an exit code being returned, which then tells the YARN to restart it.
```

Explanation: Even if the original description assumes that it contains the expected behavior and an overview, there is no clear structure through topics. The degree of completeness is therefore 3 as the original description was a plain text without any headings or structure."""
    }
]

In [3]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"], 
    template="""Bug Report: {input}

Answer: Let's think step by step.
{output}"""
)

In [4]:
bugReportStructure_examplesCoT_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix= """Bug report: {bug_report}

Answer: Let's think step by step.""",
    input_variables=["bug_report"]
)

bugReportStructure_examplesCoT_prompt.save("prompts/bugReportStructure/bugReportStructurePrompt_FewShotCoT_Examples_V1.2.0.json")

In [5]:
bugReportStructure_task_temp = """'Bug Report Structure':
The description field is important for the bug report, as it describes the issue in detail.

### Description Components ###
The description of a bug report should contain the following important components:
Overview: A description of the context/problem, a summary, the requirement, or a problem statement. This is usually the beginning of the description. Keywords for overview heading are: overview, summary, description, problem;
Steps to reproduce: A list of steps to reproduce, or the scenario to produce the error. Keywords for steps to reproduce heading are: steps, reproduce, scenario, how to;
Actual result: The result that is currently produced, the actual behaviour, or what actual happens. Keywords for actual result heading are: actual, result, behaviour, happens;
Expected result: The result that was expected, the expected behaviour, or what should happen. Keywords for expected resul heading are: expected, result, behaviour, should.

Additionally, the description could contain the following components:
Stack Traces: A list of relevant stack traces for the bug report.
Build date and hardware: The build date and hardware, as well as when and where the bug occurred.
Additional information: Additional information relevant to the bug report, including screenshots, videos, or other files.

### Task ###
We want to determine the degree of completeness of the original/old description based on the provided context of the description field.
The important components of a description are an overview, steps to reproduce, the actual result and the expected result.
We define a clear structure when we have appropriate headings for each important component in the description, except fot the overview component.
Even if the overview component does not have a heading, it is considered to be present and complete.
The degree is between 1 and 3:
1: The original/old description contains all important components or has only been slightly changed.
2: The original/old description contains 2 or 3 components but is not complete.
3: The original/old description is a plain text without a clear structure.

If the description field is missing some important parts, we want to rearrange the description field accordingly.
Return the degree of completeness, the revised description, and a short two-sentence explanation for the changes made.

The bug report is in JSON format.
{examples}",
"""

bugReportStructure_task_prompt = PromptTemplate(template=bugReportStructure_task_temp, input_variables=["examples"])

bugReportStructure_task_prompt.save("prompts/bugReportStructure/bugReportStructurePrompt_FewShotCoT_V1.3.0.json")

#### Output Format Prompt

In [4]:
bugReportStructure_format_temp = """### Instruction ###
You receive a revised bug report from an issue tracker.
Return the given data in the following format in JSON:
"<smell_degree>": "<This is the degree of change, between 1 and 3>"
"<description_new>": "<The received recommended and revised description>"
"<reason>": "<The received two sentence explanation for the change and the degree of change>"

If the description was not changed, return the original and unchanged description in the "<description_new>" field.
Don't return any source code or code snippets.

### Context ###
Data: {revised_bug_report}"""

bugReportStructure_format_prompt = PromptTemplate(template= bugReportStructure_format_temp, input_variables= ["revised_bug_report"])

bugReportStructure_format_prompt.save("prompts/bugReportStructure/bugReportStructureFormatPrompt_V2.2.1.json")